# utils

> Utility functions for CLI tools

In [ ]:
#| default_exp cli.utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import inspect
from typing import List, Tuple, Optional, Dict, Any, Callable, Union
from dataclasses import dataclass
import re
from pathlib import Path
from cjm_fasthtml_tailwind.cli.cli_config import LibraryConfig, get_active_config

## Data Structures

Common data structures used across CLI tools:

In [ ]:
#| export
@dataclass
class SearchResult:
    """Represents a single search result."""
    content_type: str  # 'factory', 'example', 'helper', 'module'
    module_name: str  # Module where found
    item_name: str  # Name of the item (factory name, function name, etc.)
    match_context: str  # The context where the match was found
    match_location: str  # Where the match was found (name, doc, source)
    score: float = 1.0  # Relevance score for fuzzy matching

## Display Helper Functions

Common helper functions for formatting display output:

In [ ]:
#| export
def print_header(
    title: str,  # TODO: Add description
    width: int = 60  # TODO: Add description
): # TODO: Add type hint
    """Print a formatted header with title and separator."""
    print(title)
    print("=" * width)
    if title.endswith(":"):
        print()  # Add extra line after colon headers

In [ ]:
#| export
def print_not_found(
    item_type: str,  # TODO: Add description
    item_name: str,  # TODO: Add description
    module_name: Optional[str] = None,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration
): # TODO: Add type hint
    """Print a standardized not found message."""
    if config is None:
        config = get_active_config()
    
    if module_name:
        print(f"No {item_type} '{item_name}' found in module '{module_name}' or module doesn't exist.")
        print(f"\nTry running '{config.cli_command} modules' to see available modules.")
    else:
        print(f"No {item_type} found in any utility modules.")

In [ ]:
#| export
def print_total(
    item_type: str,  # TODO: Add description
    count: int,  # TODO: Add description
    across_modules: bool = False  # TODO: Add description
): # TODO: Add type hint
    """Print a standardized total count message."""
    if across_modules:
        print(f"\nTotal {item_type} across all modules: {count}")
    else:
        print(f"\nTotal {item_type}: {count}")

In [ ]:
#| export
def print_helpful_instructions(
    instructions: List[Tuple[str, Optional[str]]]  # List of (description, example) tuples
):
    "Print helpful instructions section."
    print("\nTo explore further:")
    for desc, example in instructions:
        print(f"  {desc}")
        if example:
            print(f"    {example}")

In [ ]:
#| export
def display_items_generic(
    items: Union[Dict, List],  # Items to display (dict of lists or list)
    title: str,  # Title for the display
    item_formatter: Callable[[Any], str],  # Function to format each item
    item_type: str,  # Type of items for the total message
    instructions: Optional[List[Tuple[str, Optional[str]]]] = None,  # Help instructions
    not_found_message: Optional[str] = None  # Custom not found message
):
    """Generic function to display a collection of items with consistent formatting."""
    if not items:
        print(not_found_message or f"No {item_type} found.")
        return
    
    print_header(title, 60)
    
    total = 0
    if isinstance(items, dict):
        for key, values in items.items():
            if values:  # Only show if there are items
                print(f"\n{key} ({len(values)} {item_type})")
                print("-" * (len(key) + len(f" ({len(values)} {item_type})")))
                for item in values:
                    print(item_formatter(item))
                total += len(values)
    else:
        for item in items:
            print(item_formatter(item))
        total = len(items)
    
    print_total(item_type, total, across_modules=isinstance(items, dict))
    
    if instructions:
        print_helpful_instructions(instructions)

## Search Helper Functions

Helper functions for search operations:

In [ ]:
#| export
def handle_module_not_found(
    item_type: str,  # Type of items not found (e.g., 'factories', 'examples')
    module_name: str,  # Name of the module that wasn't found
    config: Optional[LibraryConfig] = None  # Optional configuration
):
    """Print standardized error message for module not found."""
    if config is None:
        config = get_active_config()
    print(f"No {item_type} found in module '{module_name}' or module doesn't exist.")
    print(f"\nTry running '{config.cli_command} modules' to see available modules.")

In [ ]:
#| export
def simple_item_formatter(
    name_field: str,  # Name of the field containing the item name
    doc_field: str  # Name of the field containing the documentation
) -> Callable[[Any], str]:  # Formatter function
    """Create a simple formatter for items with name and documentation fields."""
    def formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        name = getattr(item, name_field, "Unknown")
        doc = getattr(item, doc_field, "No documentation available")
        return f"{name}: {doc}"
    return formatter

In [ ]:
#| export
def indented_item_formatter(
    prefix: str = "  "  # Indentation prefix
) -> Callable[[Any], Callable[[Any], str]]:  # Returns a formatter factory
    """Create a formatter that indents items with a prefix."""
    def make_formatter(
        inner_formatter: Callable[[Any], str]  # TODO: Add description
    ) -> Callable[[Any], str]:  # TODO: Add return description
        "TODO: Add function description"
        def formatter(
            item  # TODO: Add type hint and description
        ): # TODO: Add type hint
            "TODO: Add function description"
            return prefix + inner_formatter(item)
        return formatter
    return make_formatter

In [ ]:
#| export
def extract_match_context(
    text: str,   # TODO: Add description
    query: str,   # TODO: Add description
    case_sensitive: bool = False,   # TODO: Add description
    context_size: int = 30  # TODO: Add description
) -> str:  # TODO: Add return description
    """Extract context around a match in text."""
    text_search = text if case_sensitive else text.lower()
    query_search = query if case_sensitive else query.lower()
    
    idx = text_search.find(query_search)
    if idx == -1:
        return text[:60] + "..."
    
    start = max(0, idx - context_size)
    end = min(len(text), idx + len(query) + context_size)
    
    prefix = "..." if start > 0 else ""
    suffix = "..." if end < len(text) else ""
    
    return prefix + text[start:end] + suffix

In [ ]:
#| export
def extract_source_line_context(
    source: str,   # TODO: Add description
    query: str,   # TODO: Add description
    case_sensitive: bool = False  # TODO: Add description
) -> str:  # TODO: Add return description
    """Extract line context for a match in source code."""
    source_search = source if case_sensitive else source.lower()
    query_search = query if case_sensitive else query.lower()
    
    idx = source_search.find(query_search)
    if idx == -1:
        return extract_match_context(source, query, case_sensitive)
    
    # Find line containing the match
    lines = source.split('\n')
    current_pos = 0
    
    for line_num, line in enumerate(lines):
        if current_pos <= idx < current_pos + len(line) + 1:
            return f"Line {line_num + 1}: {line.strip()}"
        current_pos += len(line) + 1
    
    # Fallback to context extraction
    return extract_match_context(source, query, case_sensitive)

In [ ]:
#| export
def create_search_result(
    content_type: str,  # TODO: Add description
    module_name: str,  # TODO: Add description
    item_name: str,  # TODO: Add description
    match_context: str,  # TODO: Add description
    match_location: str  # TODO: Add description
) -> SearchResult:  # TODO: Add return description
    """Create a SearchResult with standard fields."""
    return SearchResult(
        content_type=content_type,
        module_name=module_name,
        item_name=item_name,
        match_context=match_context,
        match_location=match_location
    )

In [ ]:
#| export
def search_in_text(
    query: str,  # Search query
    text: str,  # Text to search in
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> bool:  # True if match found
    """Check if query exists in text."""
    if not case_sensitive:
        return query.lower() in text.lower()
    return query in text

In [ ]:
#| export
def search_in_name_and_text(
    query: str,  # TODO: Add description
    item_name: str,  # TODO: Add description
    text: str,  # TODO: Add description
    content_type: str,  # TODO: Add description
    module_name: str,  # TODO: Add description
    text_location: str,  # TODO: Add description
    case_sensitive: bool = False  # TODO: Add description
) -> List[SearchResult]:  # TODO: Add return description
    """Search in both name and text fields, returning search results."""
    results = []
    
    # Search in name
    if search_in_text(query, item_name, case_sensitive):
        results.append(create_search_result(
            content_type=content_type,
            module_name=module_name,
            item_name=item_name,
            match_context=item_name,
            match_location='name'
        ))
    
    # Search in text (documentation, docstring, etc.)
    if search_in_text(query, text, case_sensitive):
        context = extract_match_context(text, query, case_sensitive)
        results.append(create_search_result(
            content_type=content_type,
            module_name=module_name,
            item_name=item_name,
            match_context=context,
            match_location=text_location
        ))
    
    return results

In [ ]:
#| export
def check_factory_usage_patterns(
    factory_name: str  # TODO: Add description
) -> List[str]:  # TODO: Add return description
    """Get regex patterns to match common factory usage patterns."""
    import re
    
    # Patterns to match factory usage
    patterns = [
        rf'\b{factory_name}\(',           # factory_name(
        rf'\b{factory_name}\.',           # factory_name.
        rf'\bstr\({factory_name}\b',      # str(factory_name
        rf'\b{factory_name}\[',           # factory_name[
        rf'\({factory_name}\)',           # (factory_name)
        rf'\b{factory_name}\s*=',        # factory_name =
    ]
    
    return patterns

## Enhanced Search Functions

Advanced search helper functions:

In [ ]:
#| export
def search_in_fields(
    item: Any,  # The item to search in
    query: str,  # Search query
    fields: Dict[str, Callable[[Any], str]],  # field_name -> getter function
    content_type: str,  # Type of content being searched
    module_name: str,  # Module containing the item
    item_name: str,  # Name of the item
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> List[SearchResult]:  # List of search results
    """Search for query in multiple fields of an item."""
    results = []
    
    for field_name, getter in fields.items():
        try:
            value = getter(item)
            if value and search_in_text(query, value, case_sensitive):
                context = extract_match_context(value, query, case_sensitive)
                results.append(create_search_result(
                    content_type=content_type,
                    module_name=module_name,
                    item_name=item_name,
                    match_context=context,
                    match_location=field_name
                ))
        except (AttributeError, TypeError):
            # Skip fields that can't be accessed
            pass
    
    return results

In [ ]:
#| export
def search_in_source_code(
    source: str,  # Source code to search in
    query: str,  # Search query
    content_type: str,  # Type of content being searched
    module_name: str,  # Module containing the source
    item_name: str,  # Name of the item
    case_sensitive: bool = False  # Whether to perform case-sensitive search
) -> Optional[SearchResult]:  # Search result or None
    """Search in source code and return result with line context."""
    if not source or not search_in_text(query, source, case_sensitive):
        return None
    
    context = extract_source_line_context(source, query, case_sensitive)
    return create_search_result(
        content_type=content_type,
        module_name=module_name,
        item_name=item_name,
        match_context=context,
        match_location='source'
    )

In [ ]:
#| export
def find_usage_in_items(
    target_name: str,  # Name of the target (factory/helper) to find usage for
    items: Dict[str, List[Any]],  # Dictionary of module_name -> list of items
    source_getter: Callable[[Any], str],  # Function to get source code from item
    item_type: str  # Type of items being searched (for display)
) -> List[Tuple[str, Any]]:  # List of (module_name, item) tuples
    """Find items that use a specific target (factory/helper)."""
    usage_items = []
    patterns = check_factory_usage_patterns(target_name)
    
    for module_name, module_items in items.items():
        for item in module_items:
            try:
                source = source_getter(item)
                if source and any(re.search(pattern, source) for pattern in patterns):
                    usage_items.append((module_name, item))
            except (AttributeError, TypeError):
                # Skip items where source can't be extracted
                pass
    
    return usage_items

## Module Discovery Helpers

Helper functions for module discovery and iteration:

## Command Generation Helpers

Helper functions for generating CLI commands:

In [ ]:
#| export
def get_view_command(
    content_type: str,  # Type of content ('factory', 'example', 'helper', 'module')
    module_name: str,  # Module name
    item_name: str,  # Item name (or feature name for examples)
    config: Optional[LibraryConfig] = None  # Optional configuration
) -> str:  # CLI command to view the item
    """Get the CLI command to view a specific item."""
    if config is None:
        config = get_active_config()
        
    cli_cmd = config.cli_command
    
    commands = {
        'factory': f"{cli_cmd} factory {module_name} {item_name}",
        'example': f"{cli_cmd} example {module_name} {item_name}",
        'helper': f"{cli_cmd} helper {module_name} {item_name}",
        'module': f"{cli_cmd} factories --module {module_name}"
    }
    return commands.get(content_type, "")

In [ ]:
#| export
def format_usage_examples(
    usage_items: List[Tuple[str, Any]],  # List of (module_name, item) tuples
    item_name_getter: Callable[[Any], str],  # Function to get item name
    item_type: str,  # Type of items ('examples' or 'helpers')
    view_command_type: str  # Type for get_view_command ('example' or 'helper')
) -> List[str]:  # List of formatted strings
    """Format usage examples for display."""
    formatted = []
    
    for module_name, item in usage_items:
        item_name = item_name_getter(item)
        # Extract feature name for examples
        if view_command_type == 'example' and hasattr(item, 'feature'):
            view_name = item.feature
        else:
            view_name = item_name
            
        formatted.append(f"  - {item_name} ({module_name} module)")
        formatted.append(f"    View with: {get_view_command(view_command_type, module_name, view_name)}")
    
    return formatted

In [ ]:
#| export
import importlib
import pkgutil

def discover_utility_modules(
    config: Optional[LibraryConfig] = None,  # Optional configuration, uses active if not provided
    include_submodules: bool = True  # Whether to include submodules
) -> List[Tuple[str, Any]]:  # List of (module_name, module) tuples
    """Discover all utility modules based on configuration, including submodules."""
    if config is None:
        config = get_active_config()
    
    modules = []
    
    def discover_modules_recursive(package_path: str, base_name: str = ""):
        """Recursively discover modules and submodules."""
        try:
            # Import the package
            package = importlib.import_module(package_path)
            
            # Get the package path
            if hasattr(package, '__path__'):
                # Iterate through all modules in the package
                for importer, modname, ispkg in pkgutil.iter_modules(package.__path__, 
                                                                     prefix=f'{package_path}.'):
                    try:
                        module = importlib.import_module(modname)
                        # Extract the relative name from the base discovery path
                        if base_name:
                            short_name = modname.replace(f"{config.package_name}.{base_name}.", "")
                        else:
                            short_name = modname.split('.')[-1]
                        
                        if ispkg and include_submodules:
                            # If it's a package, recursively discover its modules
                            submodules = discover_modules_recursive(modname, base_name or discovery_path)
                            modules.extend(submodules)
                        else:
                            # Add the module itself
                            modules.append((short_name, module))
                    except ImportError:
                        pass  # Skip modules that can't be imported
        except ImportError:
            pass  # Package not found
        
        return modules
    
    # Iterate through all configured discovery paths
    for discovery_path in config.module_discovery_paths:
        package_path = f"{config.package_name}.{discovery_path}"
        discovered = discover_modules_recursive(package_path, discovery_path)
        modules.extend(discovered)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_modules = []
    for name, module in modules:
        if name not in seen:
            seen.add(name)
            unique_modules.append((name, module))
    
    return sorted(unique_modules, key=lambda x: x[0])  # Sort by module name

In [ ]:
#| export
def iterate_all_modules_with_items(
    extractor_func,    # Function to extract items from a module - TODO: Add type hint
    module_filter: Optional[str] = None,  # Optional specific module to filter for
    config: Optional[LibraryConfig] = None  # Optional configuration
) -> Dict[str, List[Any]]:  # Dictionary mapping module names to their items
    """Generic iterator for extracting items from all modules."""
    if config is None:
        config = get_active_config()
        
    all_items = {}
    
    for module_name, module in discover_utility_modules(config):
        if module_filter and module_name != module_filter:
            continue
            
        items = extractor_func(module, module_name)
        if items:  # Only include modules that have items
            all_items[module_name] = items
    
    return all_items

In [ ]:
#| export
def extract_helper_names_from_test(
    source: str  # Source code of the test_<module>_helper_examples function
) -> List[str]:  # List of helper function names
    """Extract helper function names from test source code."""
    helper_names = set()
    
    # Pattern to match function calls (simple pattern for common cases)
    # Matches: function_name( or function_name (
    import_pattern = re.compile(r'from\s+[\w.]+\s+import\s+([\w\s,]+)')
    call_pattern = re.compile(r'\b(\w+)\s*\(')
    
    # First, check for explicit imports in the test
    imported_names = set()
    for match in import_pattern.finditer(source):
        imports = match.group(1)
        # Split by comma and clean up
        for name in imports.split(','):
            name = name.strip()
            if name and not name.startswith('test_'):
                imported_names.add(name)
    
    # Find all function calls
    for match in call_pattern.finditer(source):
        func_name = match.group(1)
        # Skip common built-ins and test functions
        if (func_name not in ['assert', 'print', 'str', 'int', 'float', 'list', 'dict', 
                              'tuple', 'set', 'len', 'range', 'enumerate', 'zip',
                              'isinstance', 'hasattr', 'getattr', 'setattr'] and
            not func_name.startswith('test_') and
            not func_name.startswith('_')):
            # If we have imports, only include explicitly imported names
            if imported_names:
                if func_name in imported_names:
                    helper_names.add(func_name)
            else:
                # Otherwise include all non-builtin function calls
                helper_names.add(func_name)
    
    return sorted(list(helper_names))

In [ ]:
#| export
def load_code_from_file(
    filepath: str  # TODO: Add description
) -> Optional[str]:  # TODO: Add return description
    """Load code from a file."""
    try:
        path = Path(filepath).expanduser().resolve()
        if not path.exists():
            print(f"Error: File '{filepath}' not found.")
            return None
        
        if not path.is_file():
            print(f"Error: '{filepath}' is not a file.")
            return None
            
        with open(path, 'r') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return None

In [ ]:
#| export
def list_utility_modules(
    config: Optional[LibraryConfig] = None  # Optional configuration
) -> Dict[str, str]:  # Dictionary mapping module names to their docstrings
    """List all available utility modules with their docstrings."""
    if config is None:
        config = get_active_config()
        
    modules_info = {}
    
    for module_name, module in discover_utility_modules(config):
        # Get module docstring
        doc = inspect.getdoc(module) or "No documentation available"
        modules_info[module_name] = doc
    
    return modules_info

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()